In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()#auth_mode='paste'
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=N-eJo07mrb4L6-glaDqaVqJYZ0a9gPjnTn6pga5QAQI&tc=08pypZm0G-ZZ83v2JuqBkxfJ2kikUm2CBFTZ3Z8-weM&cc=77UyRunEgcw0i7mMyiivlSY6BR-uDQBRMZvAtNZZkks

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNbsbcfPyHgJ5HNEn1WA1j83c7yEl-463ytk4NlPOdQS7XSVL2ZvYM

Successfully saved authorization token.


In [ ]:
import time

In [ ]:
geometry = ee.Geometry.Point([-71.0637, -35.0951]) #change accordingly

L8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
L8 = L8.filterBounds(geometry).filterDate('2020-01-01', '2022-11-17')

In [ ]:
def cloudFunction(img):
    prop = img.propertyNames()
    qua = img.select('QA_PIXEL')
    dilated = qua.bitwiseAnd(2).eq(0)
    cirrus = qua.bitwiseAnd(4).eq(0)
    cloud = qua.bitwiseAnd(8).eq(0)
    shadow = qua.bitwiseAnd(16).eq(0)
    snow = qua.bitwiseAnd(32).eq(0)
    mask = dilated.And(cirrus).And(cloud).And(shadow).And(snow)
    return mask.copyProperties(img, prop)

In [ ]:
L8_masks = L8.map(cloudFunction)
names = L8_masks.aggregate_array('system:index').getInfo()

In [ ]:
for n in names[:]:
    img = L8_masks.filter(ee.Filter.eq('system:index', n)).first()
    region = ee.Geometry.Polygon(ee.Geometry(ee.Image(img).get('system:footprint')).coordinates())
    ee.batch.Export.image.toDrive(img,region=region, scale=30, description=n).start()
    time.sleep(300)